In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *


In [2]:
spark = SparkSession \
    .builder \
    .appName("Projet Spark") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()


In [14]:
schema1 = StructType([StructField('Invoice_ID', StringType(), True),
                      StructField('Branch', StringType(), True),
                      StructField('City', StringType(), True),
                      StructField('Customer_type', StringType(), True),
                      StructField('Gender', StringType(), True),
                      StructField('Product_line', StringType(), True),
                      StructField('Unit_price', DoubleType(), True),
                      StructField('Quantity', DoubleType(), True),
                      StructField('Tax', DoubleType(), True),
                      StructField('Total', DoubleType(), True),
                      StructField('Date', StringType(), True),
                      StructField('Time', StringType(), True),
                      StructField('Payment', StringType(), True),
                      StructField('cogs', DoubleType(), True),
                      StructField('gross_margin_percentage', DoubleType(), True),
                      StructField('gross_income', DoubleType(), True),
                      StructField('Rating', DoubleType(), True),  
                      StructField('month_day', StringType(), True)
])

In [4]:
sales = spark.read.format('csv')\
                        .schema(schema1)\
                        .option('header', True)\
                        .option("delimiter",",")\
                        .load(r'supermarket_sales.csv')
        

In [5]:
sales = sales.withColumn("month_day",concat_ws("-",split(sales["Date"], '/').getItem(0), split(sales["Date"], '/').getItem(1)))

In [6]:
liste_dates = sales.select("month_day").distinct().collect()

In [46]:
sales_streaming = spark.readStream.format('csv')\
    .schema(schema1)\
    .option("recursiveFileLookup","true")\
    .option("header","true")\
    .csv("sales_dates/*.csv")

In [47]:
print("spark is streaming " , sales_streaming.isStreaming)

spark is streaming  True


In [48]:
maxPurchasePerHour = sales_streaming.selectExpr(
    "Total").groupBy(
      col("month_day"),
    ).sum("Total")

AnalysisException: Column 'month_day' does not exist. Did you mean one of the following? [Total];
'Aggregate ['month_day], ['month_day, sum(Total#1098) AS sum(Total)#1129]
+- Project [Total#1098]
   +- StreamingRelation DataSource(org.apache.spark.sql.SparkSession@70cd0573,csv,List(),Some(StructType(StructField(Invoice_ID,StringType,true),StructField(Branch,StringType,true),StructField(City,StringType,true),StructField(Customer_type,StringType,true),StructField(Gender,StringType,true),StructField(Product_line,StringType,true),StructField(Unit_price,DoubleType,true),StructField(Quantity,DoubleType,true),StructField(Tax,DoubleType,true),StructField(Total,DoubleType,true),StructField(Date,StringType,true),StructField(Time,StringType,true),StructField(Payment,StringType,true),StructField(cogs,DoubleType,true),StructField(gross_margin_percentage,DoubleType,true),StructField(gross_income,DoubleType,true),StructField(Rating,DoubleType,true),StructField(month_day,StringType,true))),List(),None

In [41]:
maxPurchasePerHour.writeStream\
    .format("memory")\
    .queryName("customer_table")\
    .outputMode("complete")\
    .start()

AttributeError: 'GroupedData' object has no attribute 'writeStream'